<a href="https://colab.research.google.com/github/WooHyunJ/DATATHON_9/blob/main/%EC%B5%9C%EC%A2%85_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycaret

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 95, in resolve
    result = self._result = resolver.resolve(
                            ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

# 데이터 불러오기

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/df_merge.csv")

/tmp/ipython-input-848575697.py:2: DtypeWarning: Columns (700,703,1508) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/df_merge.csv")


In [ ]:
df['SEQN'].duplicated().sum()

np.int64(9767)

In [ ]:
df = df.drop_duplicates(subset=['SEQN'], keep='first')

# 사용할 칼럼 추출

In [ ]:
#중성지방
df = df[df['LBXTR']<= 600]

#HDL 콜레스테롤
df = df[(df['LBDHDD'] >= 20) & (df['LBDHDD'] <= 90)]

# 수축기혈압: 60 ~ 250 mmHg
df = df[(df['BPXSY2'] >= 60) & (df['BPXSY2'] <= 250)]

# 이완기혈압: 40 ~ 150 mmHg
df = df[(df['BPXDI2'] >= 40) & (df['BPXDI2'] <= 150)]

#공복혈당
df = df[(df['LBXSGL'] >= 70) & (df['LBXSGL'] <= 300)]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2786 entries, 6 to 19575
Columns: 1657 entries, SEQN to WHQ520
dtypes: float64(1590), int64(28), object(39)
memory usage: 35.2+ MB


In [ ]:
print(df.isna().sum())

SEQN           0
SDDSRVYR       0
RIDSTATR       0
RIAGENDR       0
RIDAGEYR       0
            ... 
WHD140       379
WHQ150       408
WHQ030M     2547
WHQ500      2547
WHQ520      2547
Length: 1657, dtype: int64


In [ ]:
all_renames = {
    # df_labs
    'SEQN':'SEQN',
    'LBXSGL':'공복혈당',
    'LBDHDD':'HDL콜레스테롤',
    'LBXTR':'중성지방',
    'LBDLDL':'LDL콜레스테롤',
    'LBXTC':'총콜레스테롤',
    'LBXSUA':'요산',
    'LBDB12':'비타민B혈중농도',
    'PHQ020':'커피섭취',
    'PHQ030':'알콜섭취',

    'PHQ040':'감기약복용여부',
    'PHQ050':'위장약물복용여부',
    'LBXHBS':'B형간염항체',
    # df_demo
    'RIAGENDR':'성별',
    'RIDAGEYR':'나이',
    'DMDEDUC3':'교육수준_청소년',
    'DMDEDUC2':'교육수준_성인',
    'INDHHIN2':'연간가계소득',
    # df_exam
    'BPXPLS':'60초맥박수',
    'BPXPULS':'규칙적맥박수',

    'BMXBMI':'BMI지수',
    'MGDCGSZ':'악력',
    'BPXSY2':'수축기혈압',
    'BPXDI2':'이완기혈압',
    'BMXHT':'키',
    'BMXWT':'몸무게',
    'BMXWAIST': '허리둘레',
    # df_quest
    'DIQ010':'당뇨진단여부',
    'DIQ050':'혈당조절약복용여부',
    'BPQ020':'고혈압진단여부',

    'BPQ040A':'혈압약복용여부',
    'DBQ197':'유제품섭취빈도',
    'DBD910':'냉동식품섭취횟수',
    'ALQ120Q':'1년음주빈도',
    'SMQ040':'현재흡연여부',
    'SLQ050':'하루평균수면시간',
    'PAQ605':'주당운동빈도',
    'DPQ020':'우울감경험여부',
    'DPQ040':'활력수준',

    'WHQ070':'체중조절시도여부',
    'PAD680':'하루좌식시간',
    'HUQ010':'본인건강상태',
    # df_medi
    'RXDDAYS':'약복용일수',
    'RXDCOUNT':'복용한처방약수',
    'HSD010':'전반적건강상태',
    'HSQ571':'지난12개월동안_헌혈했는지',
}

In [ ]:
labs_a = list(all_renames.keys())
df_selected = df[labs_a].copy()

In [ ]:
missing_map = {
    'SEQN' : [3,4], #수치형
    'LBXSGL' : [], # 수치형
    'LBDHDD' : [], #수치형
    'LBXTR' : [], #수치형
    'LBDLDL' : [], #수치형

    'LBXTC' : [], #수치형
    'LBXSUA' : [], #수치형
    'LBDB12' : [], #수치형
    'PHQ020' : [7, 9],#범주형
    'PHQ030' : [7, 9], #범주형

    'PHQ040' : [7, 9], #범주형
    'PHQ050' : [7, 9],#범주형
    'LBXHBS' : [],#범주형
    'RIAGENDR' : [],#범주형
    'RIDAGEYR' : [],#수치형

    'DMDEDUC3' : [55,66,77, 99],#범주형
    'DMDEDUC2' : [7, 9],#범주형
    'INDHHIN2' : [77, 99],#범주형
    'BPXPLS' : [], #수치형
    'BPXPULS' : [],#범주형

    'BMXBMI' : [],#수치형
    'MGDCGSZ' : [],#수치형
    'BPXSY2' : [],#수치형
    'BPXDI2' : [],#수치형
    'BMXHT' : [],#수치형

    'BMXWT' : [],#수치형
    'BMXWAIST' : [], #연속형
    'DIQ010' : [3,7,9],#범주형
    'DIQ050' : [7,9],#범주형
    'BPQ020' : [7,9],#범주형

    'BPQ040A' : [7,9],#범주형
    'DBQ197' : [4,7,9],#범주형
    'DBD910' : [7777,9999],#연속형
    'ALQ120Q' : [777,999],#연속형
    'SMQ040' : [7,9],#범주형

    'SLQ050' : [],#범주
    'PAQ605' : [7,9],#수치형
    'DPQ020' : [7,9],#범주형
    'DPQ040' : [7,9],#범주형

    'WHQ070' : [7,9],#범주형
    'PAD680' : [7777,9999],#수치형
    'HUQ010' : [7,9], #범주형
    'RXDDAYS' : [99999], #수치형
    'RXDCOUNT' : [],#수치형

    'HSD010' : [9],#범주형
    'HSQ571' : [9],#범주형
}

for col, missing_values in missing_map.items():
    if len(missing_values) > 0:
        df_selected = df_selected[~df_selected[col].isin(missing_values)]

# KNNImputer 진행
- 연속형 변수의 결측값은 K개의 이웃의 평균값으로 대체
- 범주형 변수의 결측값은 우선 빈도수로 대체

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

# all_renames과 missing_map 정의 부분은 제외하고, 실행 코드 부분만 정리했습니다.
# (이전에 제공된 all_renames과 missing_map은 코드가 실행되기 전에 정의되어 있어야 합니다.)

# df_selected 정의 (이전 단계에서 df[labs_a].copy()를 통해 생성됨을 가정)
# all_renames = { ... }
# labs_a = list(all_renames.keys())
# df_selected = df[labs_a].copy()
# missing_map = { ... }

# --- 1. [1차 필터링] 결측치로 정의된 특정 값(7, 9, 77 등)을 가진 행 제거 ---
df_filtered = df_selected.copy()
print(f"필터링 전 행 수: {len(df_filtered)}")
for col, missing_values in missing_map.items():
    if len(missing_values) > 0:
        # isin(missing_values)에 해당하는 행을 '제외' (~)
        df_filtered = df_filtered[~df_filtered[col].isin(missing_values)]
print(f"필터링 후 행 수: {len(df_filtered)}")

# --- 2. 변수 리스트 정의 (사용자 주석 기반) ---
continuous_cols = [
    'LBXSGL', 'LBDHDD', 'LBXTR', 'LBDLDL', 'LBXTC', 'LBXSUA', 'LBDB12',
    'RIDAGEYR', 'BPXPLS', 'BMXBMI', 'MGDCGSZ', 'BPXSY2', 'BPXDI2',
    'BMXHT', 'BMXWT', 'DBD910', 'ALQ120Q', 'PAQ605',
    'PAD680', 'RXDDAYS', 'RXDCOUNT', 'BMXWAIST'
]
categorical_cols = [
    'PHQ020', 'PHQ030', 'PHQ040', 'PHQ050', 'LBXHBS',
    'RIAGENDR', 'DMDEDUC3', 'DMDEDUC2', 'INDHHIN2',
    'BPXPULS', 'DIQ010', 'DIQ050', 'BPQ020', 'BPQ040A',
    'DBQ197', 'SMQ040', 'DPQ020', 'DPQ040',
    'WHQ070', 'HUQ010', 'HSD010', 'HSQ571',
    'SLQ050'
]

# --- 3. [1차 처리] 범주형 변수 (최빈값 대체) ---
print("\n--- 3. 범주형 변수 최빈값 대체 중... ---")
df_imputed = df_filtered.copy()
for col in categorical_cols:
    if col in df_imputed.columns:
        # .mode()[0]은 가장 많이 등장한 값(최빈값)을 선택합니다.
        mode_value = df_imputed[col].mode()[0]
        df_imputed[col] = df_imputed[col].fillna(mode_value)
print("범주형 변수 처리 완료.")


# --- 4. [스케일링] 연속형 변수 (StandardScaler) ---
# 🚨🚨🚨 디버깅 코드 추가: 분산 0인 열 찾기 🚨🚨🚨
df_cont_check = df_imputed[continuous_cols]

# 분산이 0인 (즉, 값이 모두 같은) 열을 찾습니다.
zero_variance_cols = df_cont_check.loc[:, df_cont_check.apply(lambda x: x.std() == 0)].columns.tolist()

if zero_variance_cols:
    print(f"⚠️ 경고: 분산이 0인 변수가 발견되었습니다. (KNNImputer가 제외하는 원인): {zero_variance_cols}")

    # 해당 변수를 continuous_cols에서 제거합니다.
    continuous_cols = [col for col in continuous_cols if col not in zero_variance_cols]
    print(f"새로운 continuous_cols 개수: {len(continuous_cols)}")

# (이전 1~3단계 코드는 동일)

# --- 4. [스케일링] 연속형 변수 (StandardScaler) ---

# 🚨🚨🚨 디버깅 코드 추가: 분산 0인 열 찾기 🚨🚨🚨
df_cont_check = df_imputed[continuous_cols]

# 분산이 0인 (즉, 값이 모두 같은) 열을 찾습니다.
# 이 열은 StandardScaler가 오류 없이 처리하지 못하거나, 데이터에 변화가 없으므로 제외할 수 있습니다.
zero_variance_cols = df_cont_check.loc[:, df_cont_check.apply(lambda x: x.std() == 0)].columns.tolist()

# **주의: 분산이 0인 열을 continuous_cols에서 영구적으로 제거합니다.**
if zero_variance_cols:
    print(f"⚠️ 경고: 분산이 0인 변수가 발견되었습니다. (KNNImputer가 제외하는 원인): {zero_variance_cols}")

    # 해당 변수를 continuous_cols에서 제거합니다.
    continuous_cols = [col for col in continuous_cols if col not in zero_variance_cols]
    print(f"새로운 continuous_cols 개수: {len(continuous_cols)}")

# --- 4. [스케일링] 연속형 변수 (StandardScaler) ---

# 🚨🚨🚨 디버깅 코드 추가: 분산 0인 열 찾기 🚨🚨🚨
df_cont_check = df_imputed[continuous_cols]

# 분산이 0인 (즉, 값이 모두 같은) 열을 찾습니다.
zero_variance_cols = df_cont_check.loc[:, df_cont_check.apply(lambda x: x.std() == 0)].columns.tolist()

if zero_variance_cols:
    print(f"⚠️ 경고: 분산이 0인 변수가 발견되었습니다. (KNNImputer가 제외하는 원인): {zero_variance_cols}")

    # 해당 변수를 continuous_cols에서 제거합니다.
    continuous_cols = [col for col in continuous_cols if col not in zero_variance_cols]
    print(f"새로운 continuous_cols 개수: {len(continuous_cols)}")

print("--- 4. 연속형 변수 스케일링 중... ---")
original_index = df_imputed.index

# 💡 수정된 부분: StandardScaler 객체를 재정의하여 초기화합니다.
# 이전에 사용된 scaler 객체의 상태를 완전히 지우고, 새로운 fit에 대비합니다.
scaler = StandardScaler()

# 수정된 continuous_cols (22개)를 사용하여 스케일링을 진행합니다.
scaled_continuous_array = scaler.fit_transform(df_imputed[continuous_cols])

print(f"DEBUG: scaled_continuous_array의 열 개수: {scaled_continuous_array.shape[1]}")
if scaled_continuous_array.shape[1] != len(continuous_cols):
    print("⚠️ 경고: 스케일링 후 열 개수가 continuous_cols와 다릅니다!")
print("스케일링 완료.")

# --- 5. [핵심] KNNImputer 적용 (연속형 변수에만) ---
print("--- 5. KNN Imputer 적용 중... (시간이 걸릴 수 있습니다) ---")
imputer = KNNImputer(n_neighbors=5)

# 22개 열을 가진 스케일링된 NumPy 배열을 Imputer에 전달합니다.
imputed_data_scaled = imputer.fit_transform(scaled_continuous_array)

# ... (5단계 나머지 코드는 동일) ...

print("--- 6. 스케일링 역변환 중... ---")
# 이제 22개 열에 fit된 scaler로 22개 열의 데이터를 역변환합니다.
imputed_data_original = scaler.inverse_transform(imputed_data_scaled)
print("역변환 완료.")


# --- 7. [결합] 최종 데이터셋 완성 ---
print("--- 7. 최종 데이터셋 결합 중... ---")
# (1) 최빈값이 채워진 범주형 데이터
df_final_cat = df_imputed[categorical_cols]
# (2) KNN으로 예측된 연속형 데이터 (원본 스케일)
df_final_cont = pd.DataFrame(imputed_data_original,
                             columns=continuous_cols,
                             index=original_index)
# (3) ID 데이터
df_final_id = df_imputed[['SEQN']]
# (4) ID + 범주형 + 연속형 최종 결합
df_complete = pd.concat([df_final_id, df_final_cat, df_final_cont], axis=1)
# (5) 원본 컬럼 순서로 복원
df_complete = df_complete[df_filtered.columns]
print("✨ 최종 데이터셋 'df_complete' 완성.")

# --- 8. 최종 확인 ---
print("\n--- 최종 결측값 현황 (0이 나와야 함) ---")
final_na_count = df_complete.isna().sum().sum()
print(f"남아있는 총 결측값 개수: {final_na_count}")
print(f"\n--- 최종 데이터프레임 정보 ---")
print(df_complete.info())

필터링 전 행 수: 2588
필터링 후 행 수: 2588

--- 3. 범주형 변수 최빈값 대체 중... ---
범주형 변수 처리 완료.
--- 4. 연속형 변수 스케일링 중... ---
DEBUG: scaled_continuous_array의 열 개수: 22
스케일링 완료.
--- 5. KNN Imputer 적용 중... (시간이 걸릴 수 있습니다) ---
--- 6. 스케일링 역변환 중... ---
역변환 완료.
--- 7. 최종 데이터셋 결합 중... ---
✨ 최종 데이터셋 'df_complete' 완성.

--- 최종 결측값 현황 (0이 나와야 함) ---
남아있는 총 결측값 개수: 0

--- 최종 데이터프레임 정보 ---
<class 'pandas.core.frame.DataFrame'>
Index: 2588 entries, 6 to 19567
Data columns (total 46 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SEQN      2588 non-null   int64  
 1   LBXSGL    2588 non-null   float64
 2   LBDHDD    2588 non-null   float64
 3   LBXTR     2588 non-null   float64
 4   LBDLDL    2588 non-null   float64
 5   LBXTC     2588 non-null   float64
 6   LBXSUA    2588 non-null   float64
 7   LBDB12    2588 non-null   float64
 8   PHQ020    2588 non-null   float64
 9   PHQ030    2588 non-null   float64
 10  PHQ040    2588 non-null   float64
 11  PHQ050    2588 non-null   

# AutoML 진행
- 범주형 변수의 결측값은 상황에 맞는 분류모델로 예측하여 대체

In [ ]:
# (1/3) 라이브러리 설치 및 임포트
!pip install imbalanced-learn -q

import pandas as pd
import time
from sklearn.preprocessing import LabelEncoder # (기존 코드)

# ▼▼▼ 추가된 라이브러리 ▼▼▼
import numpy as np
from imblearn.over_sampling import SMOTE       # (2) 불균형 처리 (SMOTE)
from sklearn.ensemble import RandomForestClassifier # (1) AutoML (모델 추가)
from sklearn.model_selection import cross_val_score # (1) AutoML (성능 비교)
from lightgbm import LGBMClassifier
# ▲▲▲ 추가된 라이브러리 ▲▲▲


# -----------------------------------------------------------------
# (2/3) 기존 코드와 동일한 부분 (변수 정의)
# -----------------------------------------------------------------

df_final_imputed = df_complete.copy()

# 2. 수치형/연속형 변수 (22개)
continuous_cols = [
    'LBXSGL',   # 수치형
    'LBDHDD',   # 수치형
    'LBXTR',    # 수치형
    'LBDLDL',   # 수치형
    'LBXTC',    # 수치형
    'LBXSUA',   # 수치형
    'LBDB12',   # 수치형
    'RIDAGEYR', # 수치형


    'BPXPLS',   # 수치형
    'BMXBMI',   # 수치형
    'MGDCGSZ',  # 수치형
    'BPXSY2',   # 수치형
    'BPXDI2',   # 수치형
    'BMXHT',    # 수치형
    'BMXWT',    # 수치형
    'DBD910',   # 연속형
    'ALQ120Q',  # 연속형
    'PAQ605',   # 수치형
    'PAD680',   # 수치형
    'RXDDAYS',  # 수치형
    'RXDCOUNT', # 수치형
    'BMXWAIST'  # 연속형
]

# 범주형 변수 (37개)
categorical_cols = [
    'PHQ020',   # 범주형
    'PHQ030',   # 범주형
    'PHQ040',   # 범주형
    'PHQ050',   # 범주형
    'LBXHBS',   # 범주형
    'RIAGENDR', # 범주형
    'DMDEDUC3', # 범주형
    'DMDEDUC2', # 범주형
    'INDHHIN2', # 범주형
    'BPXPULS',  # 범주형
    'DIQ010',   # 범주형
    'DIQ050',   # 범주형
    'BPQ020',   # 범주형
    'BPQ040A',  # 범주형
    'DBQ197',   # 범주형
    'SMQ040',   # 범주형
    'DPQ020',   # 범주형
    'DPQ040',   # 범주형
    'WHQ070',   # 범주형
    'HUQ010',   # 범주형

    'HSD010',   # 범주형
    'HSQ571',   # 범주형
    'SLQ050',   # 범주형
]
# 18
print(f"--- 3. 범주형 변수 예측 대체(Re-Imputation) 시작 ---")
print(f"대상 변수: {len(categorical_cols)}개")
start_total_time = time.time()


# -----------------------------------------------------------------
# (3/3) AutoML 및 불균형 처리 기능이 추가된 for 루프
# -----------------------------------------------------------------

# 23 # 40개 범주형 변수 순회하며 예측
for i, y_col in enumerate(categorical_cols):

    print(f"\n[{i+1}/{len(categorical_cols)}] '{y_col}' 변수 예측 중...")
    start_col_time = time.time()

    # --- A. 원본(df_filtered)에서 결측이었던 행의 마스크(mask) 생성 ---
    original_missing_mask = df_filtered[y_col].isna()

    # --- B. 만약 원본에 결측치가 하나도 없었다면, 예측할 필요 없음 ---
    if original_missing_mask.sum() == 0:
        print(f" -> '{y_col}'는 원본 결측치가 없어 건너뜁니다.")
        continue

    # --- C. X (예측 변수) / y (타겟 변수) 정의 ---
    X_cols = [col for col in df_final_imputed.columns if col != y_col and col != 'SEQN']

    # --- D. 학습 데이터 / 예측 대상 데이터 분리 ---

    # (1) 학습 데이터 (원본에 값이 있었던 행)
    X_train = df_final_imputed.loc[~original_missing_mask, X_cols]
    y_train = df_filtered.loc[~original_missing_mask, y_col].astype(int)

    # (2) 예측 대상 데이터 (원본에 값이 없었던 행)
    X_predict = df_final_imputed.loc[original_missing_mask, X_cols]

    # (3) One-Hot Encoding (기존 코드와 동일)
    # (이 부분은 PyCaret이 아니므로 수동으로 인코딩해야 합니다)
    cat_features_for_X = [col for col in X_cols if col in categorical_cols]

    # X_train과 X_predict를 합쳐서 get_dummies를 실행 (컬럼 불일치 방지)
    X_combined = pd.concat([X_train, X_predict])
    X_combined_encoded = pd.get_dummies(X_combined, columns=cat_features_for_X, drop_first=True)

    # 다시 분리
    X_train_encoded = X_combined_encoded.iloc[:len(X_train)]
    X_predict_encoded = X_combined_encoded.iloc[len(X_train):]


    # --- E. 분류 모델 학습 및 예측 (SMOTE + 모델 비교) ---

    # E-1. (목표 2) 데이터 불균형 처리 (SMOTE)
    try:
        print("  (1/4) 데이터 불균형 처리 (SMOTE) 적용 중...")
        # (k_neighbors는 1보다 커야 하며, 가장 적은 클래스 수보다 작아야 함)
        min_class_count = y_train.value_counts().min()
        k_neighbors = max(1, min(5, min_class_count - 1)) # 오류 방지

        smote = SMOTE(random_state=42, k_neighbors=k_neighbors)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train_encoded, y_train)

    except Exception as e:
        # (샘플이 너무 적으면 SMOTE가 실패할 수 있음)
        print(f"  (!) SMOTE 적용 실패: {e} -> 원본 데이터로 진행.")
        X_train_resampled, y_train_resampled = X_train_encoded, y_train

    # E-2. (목표 1) AutoML - 모델 비교 (LGBM vs RandomForest)
    print("  (2/4) 모델 비교 (LGBM vs RandomForest)...")
    lgbm = LGBMClassifier(n_estimators=100, random_state=42, n_jobs=-1, verbose=-1)
    rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

    # 교차 검증 (cv=3)
    # (데이터가 너무 적으면 교차 검증이 불가능할 수 있음)
    n_splits = max(2, min(3, y_train_resampled.value_counts().min()))

    if n_splits < 2:
        print("  (!) 교차 검증 불가 (샘플 부족). LGBM을 기본 모델로 사용.")
        best_model = lgbm
    else:
        try:
            lgbm_score = np.mean(cross_val_score(lgbm, X_train_resampled, y_train_resampled, cv=n_splits, scoring='accuracy', n_jobs=-1))
            rf_score = np.mean(cross_val_score(rf, X_train_resampled, y_train_resampled, cv=n_splits, scoring='accuracy', n_jobs=-1))

            if lgbm_score >= rf_score:
                best_model = lgbm
                print(f"  -> 모델 선택: LightGBM (Accuracy: {lgbm_score:.4f})")
            else:
                best_model = rf
                print(f"  -> 모델 선택: RandomForest (Accuracy: {rf_score:.4f})")
        except Exception as e:
            print(f"  (!) 교차 검증 중 오류: {e} -> LGBM을 기본 모델로 사용.")
            best_model = lgbm

    # E-3. 최종 모델 학습
    print("  (3/4) 최종 모델 학습...")
    best_model.fit(X_train_resampled, y_train_resampled)

    # E-4. 예측 실행 (인코딩된 X_predict 사용)
    print("  (4/4) 결측치 예측 수행...")
    predicted_values = best_model.predict(X_predict_encoded)


    # --- F. 원본 데이터셋에 예측값 덮어쓰기 ---
    df_final_imputed.loc[original_missing_mask, y_col] = predicted_values

    col_time = time.time() - start_col_time
    print(f" -> '{y_col}' 변수 예측 완료. (소요 시간: {col_time:.2f}초)")

# -----------------------------------------------------------------
# (기존 코드와 동일) 최종 결과 확인
# -----------------------------------------------------------------

# 93 # --- 2. 최종 결과 확인 ---
total_time = time.time() - start_total_time
print(f"\n--- ✨ 모든 범주형 변수 예측 대체 완료! ---")
print(f"총 소요 시간: {total_time:.2f}초")

# 98 # 최종 결측값 확인 (0이 나와야 함)
final_na_count = df_final_imputed.isna().sum().sum()
print(f"\n최종 결측값 개수: {final_na_count}")

# 103 # 예시로 PHQ020 컬럼의 값 분포 비교
print("\n--- (예시) PHQ020 값 분포 비교 ---")
print(f"(1) KNNImputer + 최빈값 대체 (df_complete):")
print(df_complete['PHQ020'].value_counts(normalize=True).sort_index())
print(f"\n(2) KNNImputer + 모델 예측 대체 (df_final_imputed):")
print(df_final_imputed['PHQ020'].value_counts(normalize=True).sort_index())

--- 3. 범주형 변수 예측 대체(Re-Imputation) 시작 ---
대상 변수: 23개

[1/23] 'PHQ020' 변수 예측 중...
 -> 'PHQ020'는 원본 결측치가 없어 건너뜁니다.

[2/23] 'PHQ030' 변수 예측 중...
 -> 'PHQ030'는 원본 결측치가 없어 건너뜁니다.

[3/23] 'PHQ040' 변수 예측 중...
 -> 'PHQ040'는 원본 결측치가 없어 건너뜁니다.

[4/23] 'PHQ050' 변수 예측 중...
 -> 'PHQ050'는 원본 결측치가 없어 건너뜁니다.

[5/23] 'LBXHBS' 변수 예측 중...
  (1/4) 데이터 불균형 처리 (SMOTE) 적용 중...
  (2/4) 모델 비교 (LGBM vs RandomForest)...
  -> 모델 선택: RandomForest (Accuracy: 0.7749)
  (3/4) 최종 모델 학습...
  (4/4) 결측치 예측 수행...
 -> 'LBXHBS' 변수 예측 완료. (소요 시간: 13.68초)

[6/23] 'RIAGENDR' 변수 예측 중...
 -> 'RIAGENDR'는 원본 결측치가 없어 건너뜁니다.

[7/23] 'DMDEDUC3' 변수 예측 중...
  (1/4) 데이터 불균형 처리 (SMOTE) 적용 중...
  (2/4) 모델 비교 (LGBM vs RandomForest)...
  -> 모델 선택: LightGBM (Accuracy: 0.7390)
  (3/4) 최종 모델 학습...
  (4/4) 결측치 예측 수행...
 -> 'DMDEDUC3' 변수 예측 완료. (소요 시간: 17.42초)

[8/23] 'DMDEDUC2' 변수 예측 중...
  (1/4) 데이터 불균형 처리 (SMOTE) 적용 중...
  (2/4) 모델 비교 (LGBM vs RandomForest)...
  -> 모델 선택: RandomForest (Accuracy: 0.6131)
  (3/4) 최종 모델 학습...
  (4/4) 결측치 예측 수행...

/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(


  -> 모델 선택: RandomForest (Accuracy: 0.7652)
  (3/4) 최종 모델 학습...
  (4/4) 결측치 예측 수행...
 -> 'SLQ050' 변수 예측 완료. (소요 시간: 3.93초)

--- ✨ 모든 범주형 변수 예측 대체 완료! ---
총 소요 시간: 213.88초

최종 결측값 개수: 0

--- (예시) PHQ020 값 분포 비교 ---
(1) KNNImputer + 최빈값 대체 (df_complete):
PHQ020
1.0    0.007342
2.0    0.992658
Name: proportion, dtype: float64

(2) KNNImputer + 모델 예측 대체 (df_final_imputed):
PHQ020
1.0    0.007342
2.0    0.992658
Name: proportion, dtype: float64


# 데이터칼럼 라벨링

In [ ]:
# 2. 실제로 사용할 60개 변수 목록 (missing_map의 키 목록)
target_columns = [
    'SEQN', 'LBXSGL', 'LBDHDD', 'LBXTR', 'LBDLDL', 'LBXTC', 'LBXSUA', 'LBDB12',
    'PHQ020', 'PHQ030', 'PHQ040', 'PHQ050','LBXHBS',
    'RIAGENDR', 'RIDAGEYR', 'DMDEDUC3', 'DMDEDUC2',
    'INDHHIN2', 'BPXPLS', 'BPXPULS', 'BMXBMI', 'MGDCGSZ',
    'BPXSY2', 'BPXDI2', 'BMXHT', 'BMXWT', 'DIQ010', 'DIQ050', 'BPQ020',
    'BPQ040A', 'DBQ197', 'DBD895', 'DBD910', 'ALQ101', 'SMQ040', 'SLQ050',
    'PAQ605', 'DPQ020', 'DPQ040', 'WHQ500', 'PAD680', 'HUQ010',
    'RXDDAYS', 'RXDCOUNT', 'HSD010',
    'HSQ571','BMXWAIST'
]

# 3. 60개 목록에 포함된 변수만 all_renames 딕셔너리에서 추출하여
#    최종 rename 맵을 생성합니다.
final_rename_map = {
    col_eng: all_renames.get(col_eng, col_eng) # 만약 all_renames에 한글 이름이 없으면 영어 이름 그대로 사용
    for col_eng in target_columns
}

# 4. (선택 사항) 최종 rename 맵 확인
# print(final_rename_map)

# 5. (★중요) 최종 데이터프레임(df_final_imputed)에 적용하기
df_final_imputed = df_final_imputed.rename(columns=final_rename_map)

# 6. 한글 컬럼명으로 잘 바뀌었는지 확인
df_final_imputed.head()

,SEQN,공복혈당,HDL콜레스테롤,중성지방,LDL콜레스테롤,총콜레스테롤,요산,비타민B혈중농도,커피섭취,알콜섭취,...,주당운동빈도,우울감경험여부,활력수준,WHQ070,하루좌식시간,본인건강상태,약복용일수,복용한처방약수,전반적건강상태,지난12개월동안_헌혈했는지
6,73559,183.0,60.0,51.0,56.0,126.0,5.7,732.0,2.0,2.0,...,2.0,0.0,0.0,2.0,300.0,3,365.0,5.0,3.0,2.0
12,73561,104.0,85.0,75.0,101.0,201.0,4.2,225.0,2.0,2.0,...,2.0,1.0,3.0,2.0,480.0,5,152.0,4.0,5.0,2.0
24,73564,107.0,58.0,64.0,97.0,168.0,5.1,668.0,2.0,2.0,...,2.0,1.0,1.0,1.0,60.0,3,1825.0,3.0,3.0,2.0
38,73574,89.0,53.0,14.0,75.0,131.0,2.7,335.0,2.0,2.0,...,2.0,0.0,0.0,2.0,480.0,1,159.6,1.8,1.0,2.0
42,73576,83.0,61.0,57.0,82.0,154.0,5.4,619.2,2.0,2.0,...,2.0,0.0,0.0,2.0,360.0,1,979.4,1.4,1.0,2.0


# Target Y (대사증후군 진단) 제작

In [ ]:
import numpy as np

# --- 가정 (NHANES 데이터 표준 코드 기준) ---
# 1. 성별 ('RIAGENDR'): 1 = 남성, 2 = 여성
# 2. 혈압약복용여부 ('BPQ040A'): 1 = 예
# 3. 혈당조절약복용여부 ('DIQ050'): 1 = 예
# ----------------------------------------------

# 복사본을 만들어 원본을 보존합니다.
df_classified = df_final_imputed.copy()

# --- 1. 복부비만 (허리둘레) ---
cond_waist = (
    ((df_classified['성별'] == 1) & (df_classified['허리둘레'] >= 90)) |
    ((df_classified['성별'] == 2) & (df_classified['허리둘레'] >= 85))
)

df_classified['대사_복부비만'] = cond_waist.astype(int)


# --- 2. 상승된 혈압 ---
cond_bp = (
    (df_classified['수축기혈압'] >= 130) |
    (df_classified['이완기혈압'] >= 85) |
    (df_classified['혈압약복용여부'] == 1)
)
df_classified['대사_혈압'] = cond_bp.astype(int)

# --- 3. 상승된 혈당 ---
cond_glucose = (
    (df_classified['공복혈당'] >= 100) |
    (df_classified['혈당조절약복용여부'] == 1)
)
df_classified['대사_혈당'] = cond_glucose.astype(int)

# --- 4. 저하된 HDL 콜레스테롤 ---
# ⚠️ 참고: '고지혈증약 복용' 여부 변수가 없어 수치로만 판별합니다.
cond_hdl = (
    ((df_classified['성별'] == 1) & (df_classified['HDL콜레스테롤'] < 40)) |
    ((df_classified['성별'] == 2) & (df_classified['HDL콜레스테롤'] < 50))
)
df_classified['대사_HDL'] = cond_hdl.astype(int)

# --- 5. 상승된 중성지방 ---
# ⚠️ 참고: '고중성지방혈증 치료약 복용' 여부 변수가 없어 수치로만 판별합니다.
cond_tg = (df_classified['중성지방'] >= 150)
df_classified['대사_중성지방'] = cond_tg.astype(int)


# --- 최종 판정 (5개 기준) ---

# 4개 기준 중 해당되는 항목의 개수를 합산합니다.
# (만약 '허리둘레'를 추가했다면 + df_classified['대사_복부비만'] 을 더해야 합니다)
df_classified['대사증후군_위험요인_수'] = (
    df_classified['대사_복부비만'] +
    df_classified['대사_혈압'] +
    df_classified['대사_혈당'] +
    df_classified['대사_HDL'] +
    df_classified['대사_중성지방']
)

# 위험요인 개수가 3개 이상이면 '위험군', 아니면 '일반군'
threshold = 3
df_classified['대사증후군_판정'] = np.where(
    df_classified['대사증후군_위험요인_수'] >= threshold,
    '위험군',
    '일반군'
)

In [ ]:
# 1. 새로 생성된 컬럼들 확인
check_cols = [
    '대사_혈압',
    '대사_혈당',
    '대사_HDL',
    '대사_중성지방',
    '대사증후군_위험요인_수',
    '대사증후군_판정'
]
print(df_classified[check_cols].head())

# 2. 최종 판정 결과 분포 확인
print("\n--- 최종 판정 결과 (5개 기준 중 3개 이상) ---")
print(df_classified['대사증후군_판정'].value_counts())

    대사_혈압  대사_혈당  대사_HDL  대사_중성지방  대사증후군_위험요인_수 대사증후군_판정
6       1      1       0        0             3      위험군
12      1      1       0        0             3      위험군
24      1      1       0        0             3      위험군
38      1      0       0        0             1      일반군
42      0      0       0        0             0      일반군

--- 최종 판정 결과 (5개 기준 중 3개 이상) ---
대사증후군_판정
일반군    1637
위험군     951
Name: count, dtype: int64


# 좌식시간 그룹핑

In [ ]:
import numpy as np

# --- 1. 유효한 데이터 필터링 (1200분 기준) ---
# 0분 ~ 1200분(20시간) 사이의 유효한 값만 선택
df_cleaned = df_classified[
    (df_classified['하루좌식시간'] >= 0) &
    (df_classified['하루좌식시간'] <= 1200)  # <-- 1440에서 1200으로 수정
].copy()

print(f"원본 데이터 수: {len(df_classified)}")
print(f"오류 값(1200분 초과/0분 미만) 제거 후 데이터 수: {len(df_cleaned)}")


# --- 2. 2시간(120분) 단위로 그룹핑 (최대 20시간) ---
# 2시간(120분) 단위로 구간을 나눌 기준(bins)을 정의
# [0, 120, 240, ..., 1200]
bins = list(range(0, 1200 + 120, 120))  # <-- 1440에서 1200으로 수정

# 각 구간에 붙일 라벨(labels)을 숫자로 정의
# [2, 4, 6, ..., 20]
labels = [hour for hour in range(2, 20 + 1, 2)] # <-- 24에서 20으로 수정

# '하루좌식시간_그룹' 이라는 새 컬럼을 만듭니다.
df_cleaned['하루좌식시간_그룹'] = pd.cut(
    df_cleaned['하루좌식시간'],
    bins=bins,
    labels=labels,
    right=False,
    include_lowest=True
)

print("\n--- 2시간 단위 그룹핑 결과 (상위 5개) ---")
print(df_cleaned[['하루좌식시간', '하루좌식시간_그룹']].head())

print("\n--- 그룹별 인원 수 ---")
print(df_cleaned['하루좌식시간_그룹'].value_counts().sort_index())

원본 데이터 수: 2588
오류 값(1200분 초과/0분 미만) 제거 후 데이터 수: 2587

--- 2시간 단위 그룹핑 결과 (상위 5개) ---
    하루좌식시간 하루좌식시간_그룹
6    300.0         6
12   480.0        10
24    60.0         2
38   480.0        10
42   360.0         8

--- 그룹별 인원 수 ---
하루좌식시간_그룹
2      54
4     312
6     459
8     405
10    729
12    365
14    178
16     56
18     22
20      7
Name: count, dtype: int64


In [ ]:
def income_group(code):
    if code in [1, 2, 3, 4, 5, 13]:
        return "저소득층"
    elif code in [6, 7, 8, 9, 10, 12]:
        return "중간층"
    elif code in [14, 15]:
        return "고소득층"
    else:
        return np.nan

df_cleaned['소득_3단계'] = df_cleaned['연간가계소득'].apply(income_group)

In [ ]:
df_cleaned['high_sedentary'] = (df_cleaned['하루좌식시간'] >= 600).astype(int)

df_cleaned['좌식군'] = df_cleaned['high_sedentary'].map({
    0: '일반좌식군',
    1: '고위험좌식군'
})

In [ ]:
column_mapping = {
    'ALQ120Q': '1년 음주 빈도',
    'WHQ070': '체중조절 시도 여부',
}
df_cleaned.rename(columns=column_mapping, inplace=True)

In [ ]:
df_cleaned.to_csv('final_csv11.csv', index = False, encoding='utf-8-sig')